In [141]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel, polynomial_kernel
import os
import sys
maindir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(maindir)
from source.utils import make_symmetric_matrix_psd, normalize

In [52]:
def get_kernel_paramter(kernel_parameter):
    """ Returns the combination of kernel parameters from the results of the
        multitask-SVM approach based on the highest mean AUC.

        see results of the Parser_multitask_SVM.py module

        Args: kernel_parameter: results of the multitask-SVM approach as .csv file

        Returns:
            pam (list): Combination of kernel parameter for the combination of kernel functions for the
            multitask-SVM classifier
            based on the highest mean AUC value
            best_AUC (float): highest mean AUC value
        """
    pam_roc_auc = kernel_parameter[kernel_parameter['scoring'].isin(['roc_auc'])]
    pam = pam_roc_auc['best_params']
    #print(pam)
    pam = pam.str.split(" ", expand=True)
    #print(pam)
    return pam

In [17]:
def select_time_point(kernel_parameter, time_point):
    """
    Selection of the time point to run ESPY measurment
    
    Parameter:
    ---------
    kernel_parameter: dataframe
        performance results per time point
    time_point: str
        preferable time point 
        
    Returns:
    --------
    X: np.darray
        matrix of performance scores per time point
    
    """
    X = kernel_parameter[kernel_parameter['time'].isin([time_point])]
    #print(X)
    return X

In [6]:
# kernel parameter
# via pandas as csv file 
input_file = "/Users/schmidtj/Documents/GitHub/MalariaVaccineEfficacyPrediction/results/multitaskSVM/whole/RRR/unscaled/RGSCV/RepeatedGridSearchCV_results_24.03.2022_20-01-36.tsv"
results = pd.read_csv(input_file, delimiter = "\t", header=0, index_col=0)

In [132]:
input_file2 = "/Users/schmidtj/Documents/GitHub/MalariaVaccineEfficacyPrediction/results/multitaskSVM/whole/RPR/unscaled/RGSCV/RepeatedGridSearchCV_results_24.03.2022_19-28-57.tsv"
results_RPR = pd.read_csv(input_file2, delimiter = "\t", header=0, index_col=0)

In [7]:
results

,time,scoring,best_params,best_score
0,III14,mcc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.747156
1,III14,precision_recall_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.926416
2,III14,roc_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.916414
3,C-1,mcc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.778919
4,C-1,precision_recall_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.940606
5,C-1,roc_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.939141
6,C28,mcc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.854362
7,C28,precision_recall_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.943003
8,C28,roc_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.944066


In [133]:
results_RPR

,time,scoring,best_params,best_score
0,III14,mcc,"{'dataselector__P1': 5.0, 'dataselector__P2': ...",0.758499
1,III14,precision_recall_auc,"{'dataselector__P1': 3.0, 'dataselector__P2': ...",0.928810
2,III14,roc_auc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.922980
3,C-1,mcc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.799873
4,C-1,precision_recall_auc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.944031
5,C-1,roc_auc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.940909
6,C28,mcc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.785277
7,C28,precision_recall_auc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.945912
8,C28,roc_auc,"{'dataselector__P1': 2.0, 'dataselector__P2': ...",0.941667


In [13]:
time_point_III14 = select_time_point(results, "III14")

In [14]:
time_point_III14

,time,scoring,best_params,best_score
0,III14,mcc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.747156
1,III14,precision_recall_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.926416
2,III14,roc_auc,"{'dataselector__P1': 'X', 'dataselector__P2': ...",0.916414


In [134]:
time_point_III14_RPR = select_time_point(results_RPR, 'III14')

In [135]:
param_combi_RPR = get_kernel_paramter(time_point_III14_RPR)

In [136]:
param_combi_RRR = get_kernel_paramter(time_point_III14)

In [137]:
param_combi_RRR

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
2,{'dataselector__P1':,"'X',",'dataselector__P2':,"'X',",'dataselector__R0':,"0.1,",'dataselector__R1':,"0.1,",'dataselector__R2':,"1e-05,",'dataselector__SA':,"'X',",'dataselector__SO':,"'X',",'svc__C':,10.0}


In [138]:
param_combi_RPR

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
2,{'dataselector__P1':,"2.0,",'dataselector__P2':,"'X',",'dataselector__R0':,"0.01,",'dataselector__R1':,"'X',",'dataselector__R2':,"1e-06,",'dataselector__SA':,"'X',",'dataselector__SO':,"'X',",'svc__C':,0.001}


In [28]:
input_matrix = "/Users/schmidtj/Documents/GitHub/MalariaVaccineEfficacyPrediction/data/precomputed_multitask_kernels/scaled/kernel_matrix_RRR_SA_X_SO_X_R0_1.0E-01_R1_1.0E-01_R2_1.0E-05_P1_X_P2_X.npy"
kernel_matrix = np.load(input_matrix)

In [29]:
kernel_matrix

array([[ 1.        , -0.00734674, -0.07170844, ...,  0.02046826,
        -0.09263804, -0.00766244],
       [-0.00734674,  1.        ,  0.24667621, ..., -0.06584964,
        -0.19285758, -0.07305818],
       [-0.07170844,  0.24667621,  1.        , ..., -0.07863947,
        -0.20547373, -0.08677497],
       ...,
       [ 0.02046826, -0.06584964, -0.07863947, ...,  1.        ,
         0.0524356 ,  0.12904982],
       [-0.09263804, -0.19285758, -0.20547373, ...,  0.0524356 ,
         1.        ,  0.08276147],
       [-0.00766244, -0.07305818, -0.08677497, ...,  0.12904982,
         0.08276147,  1.        ]])

In [30]:
input_matrix2 = "/Users/schmidtj/Documents/GitHub/MalariaVaccineEfficacyPrediction/data/precomputed_multitask_kernels/unscaled/kernel_matrix_RRR_SA_X_SO_X_R0_1.0E-01_R1_1.0E-01_R2_1.0E-05_P1_X_P2_X.npy"
kernel_matrix2 = np.load(input_matrix2)

In [35]:
def is_equal(m1, m2):
    if (m1.all()==m2.all()):
        print('true')
    else:
        print('false')

In [36]:
is_equal(kernel_matrix, kernel_matrix2)

true


In [39]:
input_target_label = "/Users/schmidtj/Documents/GitHub/MalariaVaccineEfficacyPrediction/data/precomputed_multitask_kernels/unscaled/target_label_vec.csv"
target_label = pd.read_csv(input_target_label, index_col=0)

In [40]:
target_label

,Patient,group,Protection,Dose,TimePointOrder
0,T2-002 III 14,2,1,1,2
1,T2-005 III 14,5,0,0,2
2,T2-006 III 14,6,1,1,2
3,T2-007 III 14,7,0,1,2
4,T2-010 III 14,10,0,2,2
...,...,...,...,...,...
115,T2-067 C 28,67,1,2,4
116,T2-068 C 28,68,0,0,4
117,T2-071 C 28,71,1,3,4
118,T2-072 C 28,72,1,3,4


In [41]:
y_label = target_label.loc[:, 'Protection'].to_numpy()

In [42]:
y_label

array([1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1])

In [155]:
def multitask_model(kernel_matrix, kernel_parameter, y_label):
    """Set up multitask-SVM model based on the output of file of the rgscv_multitask.py.

    set up multitask-SVM model based on evaluated kernel combinations from Parser_multitask_SVM.py module.

    Args: data (matrix): n x m matrix (n = samples as rows, m = features as columns)
          kernel_parameter (dictionary): combination of kernel parameter

    Returns: multitaskModel (SVM model): classification model based on the multitask-SVM approach

    """

    # extract cost value C 
    #print(kernel_parameter[15])
    C_reg = pd.to_numeric(kernel_parameter[15].str.split("}", expand=True)[0])
    #print(C_reg)

    # set up multitask model based on evaluated parameter
    multitaskModel = svm.SVC(kernel="precomputed", 
                             C=C_reg,
                             probability=True,
                             random_state=1337,
                             cache_size=500)
    multitaskModel.fit(kernel_matrix, y_label)

    return multitaskModel

In [131]:
multitask_model = multitask_model(kernel_matrix, param_combi, y_label)

In [102]:
def make_feature_combination(data, upperValue, lowerValue):
    """Generate vector of feature combination.

        Generate for each single feature a vector based on Upper- and LowerQuantile value

        Args: data (matrix): matrix of proteome data, n x m matrix (n = samples as rows, m = features as columns)
              upperValue (int): value of upper quantile
              lowerValue (int): value of lower quantile

        Returns: feature_comb (matrix): combination of features
                 get_features_comb (series):  series of feature combinations
        """
    # get start point of antibody reactivity signals in data
    feature_start = data.columns.get_loc("TimePointOrder") + 1
    # get matrix of feature values
    features = data[data.columns[feature_start:]]

    # generate feature combination based on median signal intensity, intensity for upper quantile and lower quantile
    # x^p = {x1, x2, ... , x_j-1, x^p_j, X_j+1, ...., x_m}^T
    feature_comb = features.median().to_frame(name = "Median")
    feature_comb["UpperQuantile"] = features.quantile(upperValue/100)
    feature_comb["LowerQuantile"] = features.quantile(lowerValue/100)
    feature_comb = feature_comb.T
    feature_comb_arr = feature_comb.values.copy()

    # concatenate feature combinations in series
    get_features_comb = []
    get_features_comb.append(feature_comb_arr[0])
    for i in range(len(feature_comb_arr[0])):
        temp1 = feature_comb_arr[0].copy()
        temp1[i] = feature_comb_arr[1][i]
        get_features_comb.append(temp1)

        temp2 = feature_comb_arr[0].copy()
        temp2[i] = feature_comb_arr[2][i]
        get_features_comb.append(temp2)

    return feature_comb, get_features_comb


In [103]:
proteome_input_file = "/Users/schmidtj/Documents/GitHub/MalariaVaccineEfficacyPrediction/data/proteome_data/preprocessed_whole_data.csv"
proteome_data_whole = pd.read_csv(proteome_input_file)

In [104]:
proteome_data_whole

,Patient,group,Protection,Dose,TimePointOrder,mal_mito_3.iso1.exon1.amp1_4066,mal_mito_2.iso1.exon1.amp1_1466,mal_mito_1.iso1.exon1.amp1_3777,PFL2430c_484,PFL2215w_4059,...,MAL8P1.320_1o1_4243,MAL8P1.156_187,MAL8P1.155_2o3_4890,MAL8P1.143_3o15_2092,MAL8P1.143_1o15_2416,MAL8P1.143_14o15_1450,MAL8P1.140_4049,MAL7P1.32_5037,MAL13P1.303e4s1_1829,MAL13P1.303e1s1_4421
0,T2-002 C -1,2,1,1,3,6.727920,0.000000,6.409391,6.768184,0.000000,...,0.000000,0.000000,0.000000,10.651052,8.918863,0.000000,6.569856,14.609121,7.679480,2.807355
1,T2-002 C 28,2,1,1,4,6.348728,6.954196,6.238405,10.194141,10.168045,...,9.016808,9.658211,9.852530,11.159556,10.738937,10.656425,8.092757,15.251630,10.368506,9.494856
2,T2-002 III 14,2,1,1,2,7.033423,6.357552,6.954196,3.700440,0.000000,...,0.000000,0.000000,0.000000,11.168672,9.628446,0.000000,0.000000,15.327447,0.000000,0.000000
3,T2-005 C -1,5,0,0,3,9.262095,9.241983,9.487840,6.539159,9.579316,...,8.308339,9.697836,9.267957,8.409391,9.208234,8.105909,8.643856,9.227616,9.048487,7.055282
4,T2-005 C 28,5,0,0,4,7.734710,7.891784,7.954196,13.478011,8.794416,...,7.471675,8.873444,9.080818,9.632995,6.339850,8.233620,10.639793,15.874609,8.668885,8.531381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,T2-072 C 28,72,1,3,4,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,T2-072 III 14,72,1,3,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
117,T2-073 C -1,73,1,3,3,0.000000,0.000000,0.000000,0.000000,5.066089,...,0.000000,9.642052,0.000000,0.000000,0.000000,0.000000,0.000000,4.044394,0.000000,0.000000
118,T2-073 C 28,73,1,3,4,0.000000,0.000000,0.000000,0.000000,7.422065,...,8.126704,9.192293,7.654636,5.321928,6.321928,0.000000,0.000000,9.022368,0.000000,0.000000


In [106]:
np.all(np.isin(proteome_data_whole.dtypes.to_list()[5:], ['float64']))

True

In [107]:
def sort(
    data: pd.DataFrame,
) -> pd.DataFrame:
    """ Sorting

    Input data is sorted by time point to keep same patient over all four time points in order.

    Parameters
    ----------
    data : pd.DataFrame
        Raw proteome data, n x m pd.DataFrame (n = samples as rows, m = features as columns)

    Returns
    -------
    data : pd.DataFrame
        Returns sorted DataFrame
    """
    data.sort_values(by=["TimePointOrder","Patient"], inplace=True)
    data.reset_index(inplace = True)
    data.drop(columns = ['index'], inplace=True)
    return data

In [108]:
proteome_data_whole = sort(proteome_data_whole)

In [109]:
proteome_data_whole

,Patient,group,Protection,Dose,TimePointOrder,mal_mito_3.iso1.exon1.amp1_4066,mal_mito_2.iso1.exon1.amp1_1466,mal_mito_1.iso1.exon1.amp1_3777,PFL2430c_484,PFL2215w_4059,...,MAL8P1.320_1o1_4243,MAL8P1.156_187,MAL8P1.155_2o3_4890,MAL8P1.143_3o15_2092,MAL8P1.143_1o15_2416,MAL8P1.143_14o15_1450,MAL8P1.140_4049,MAL7P1.32_5037,MAL13P1.303e4s1_1829,MAL13P1.303e1s1_4421
0,T2-002 III 14,2,1,1,2,7.033423,6.357552,6.954196,3.700440,0.000000,...,0.000000,0.000000,0.000000,11.168672,9.628446,0.000000,0.000000,15.327447,0.000000,0.000000
1,T2-005 III 14,5,0,0,2,8.596190,8.481799,8.840778,9.076816,9.045760,...,8.663558,0.000000,9.177420,9.353147,7.632995,0.000000,8.290019,8.556506,8.873444,8.961450
2,T2-006 III 14,6,1,1,2,9.838416,9.589651,10.115694,10.676398,11.304922,...,10.879966,10.665336,10.838416,10.771489,10.681678,10.901621,10.989394,13.195757,11.017504,10.706064
3,T2-007 III 14,7,0,1,2,6.965784,7.451211,7.149747,10.499846,5.066089,...,0.000000,7.396605,0.000000,11.946724,8.339850,8.497852,0.000000,15.761292,6.209453,0.000000
4,T2-010 III 14,10,0,2,2,6.483816,7.977280,7.800900,10.444497,9.814582,...,8.147205,10.837628,10.098032,11.304351,10.863799,10.425740,10.063395,14.122383,10.745674,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,T2-067 C 28,67,1,2,4,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
116,T2-068 C 28,68,0,0,4,10.856426,10.606868,10.560333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.589300,0.000000,0.000000
117,T2-071 C 28,71,1,3,4,0.000000,0.000000,0.000000,9.987264,0.000000,...,0.000000,9.516685,5.303781,12.148477,0.000000,2.459432,0.000000,13.625366,0.000000,6.882643
118,T2-072 C 28,72,1,3,4,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [111]:
data_t2 = proteome_data_whole.loc[proteome_data_whole["TimePointOrder"] == 2]

In [112]:
data_t2

,Patient,group,Protection,Dose,TimePointOrder,mal_mito_3.iso1.exon1.amp1_4066,mal_mito_2.iso1.exon1.amp1_1466,mal_mito_1.iso1.exon1.amp1_3777,PFL2430c_484,PFL2215w_4059,...,MAL8P1.320_1o1_4243,MAL8P1.156_187,MAL8P1.155_2o3_4890,MAL8P1.143_3o15_2092,MAL8P1.143_1o15_2416,MAL8P1.143_14o15_1450,MAL8P1.140_4049,MAL7P1.32_5037,MAL13P1.303e4s1_1829,MAL13P1.303e1s1_4421
0,T2-002 III 14,2,1,1,2,7.033423,6.357552,6.954196,3.700440,0.000000,...,0.000000,0.000000,0.000000,11.168672,9.628446,0.000000,0.000000,15.327447,0.000000,0.000000
1,T2-005 III 14,5,0,0,2,8.596190,8.481799,8.840778,9.076816,9.045760,...,8.663558,0.000000,9.177420,9.353147,7.632995,0.000000,8.290019,8.556506,8.873444,8.961450
2,T2-006 III 14,6,1,1,2,9.838416,9.589651,10.115694,10.676398,11.304922,...,10.879966,10.665336,10.838416,10.771489,10.681678,10.901621,10.989394,13.195757,11.017504,10.706064
3,T2-007 III 14,7,0,1,2,6.965784,7.451211,7.149747,10.499846,5.066089,...,0.000000,7.396605,0.000000,11.946724,8.339850,8.497852,0.000000,15.761292,6.209453,0.000000
4,T2-010 III 14,10,0,2,2,6.483816,7.977280,7.800900,10.444497,9.814582,...,8.147205,10.837628,10.098032,11.304351,10.863799,10.425740,10.063395,14.122383,10.745674,0.000000
5,T2-011 III 14,11,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,T2-012 III 14,12,0,1,2,7.027906,7.499846,7.974415,9.063395,0.000000,...,7.388017,8.445015,9.463524,10.251482,0.000000,9.923327,0.000000,12.406736,8.828136,8.560333
7,T2-015 III 14,15,1,1,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7.409391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,T2-017 III 14,17,1,3,2,0.000000,0.000000,0.000000,0.000000,3.584963,...,7.751544,6.820179,8.197217,6.651052,4.459432,7.422065,7.434628,7.312883,5.870365,7.266787
9,T2-018 III 14,18,1,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.310613,0.000000,0.000000,0.000000,0.000000,0.000000,8.618386,0.000000,0.000000,0.000000


In [113]:
combinations, all_feature_combinations = make_feature_combination(data_t2, 75, 25)

In [117]:
combinations

,mal_mito_3.iso1.exon1.amp1_4066,mal_mito_2.iso1.exon1.amp1_1466,mal_mito_1.iso1.exon1.amp1_3777,PFL2430c_484,PFL2215w_4059,PFL1550w_501,PFL0800c_2443,PFL0630w_2120,PFL0300c_1466,PFL0210c_2437,...,MAL8P1.320_1o1_4243,MAL8P1.156_187,MAL8P1.155_2o3_4890,MAL8P1.143_3o15_2092,MAL8P1.143_1o15_2416,MAL8P1.143_14o15_1450,MAL8P1.140_4049,MAL7P1.32_5037,MAL13P1.303e4s1_1829,MAL13P1.303e1s1_4421
Median,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.935182,0.000000,2.660964,0.000000,0.000000,0.00000,5.442467,0.000000,0.00000
UpperQuantile,6.567907,6.823071,6.132035,9.212667,8.187487,6.81392,7.689942,7.549557,7.040742,4.444904,...,7.494929,8.664890,5.807655,10.196402,7.972876,7.170138,7.55249,10.682082,7.385065,7.03654
LowerQuantile,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000


In [147]:
def feature_gram_matrix(data, kernel_parameter):
    """Define feature

            Define feature for prediction in multitask-SVM classification model

            Args: data (matrix): matrix of feature (n= samples, m= feature)
                  kernel_parameter (dictionary): combination of kernel parameter

            Returns: test_sample (matrix): matrix of defined feature

            """
    # kernel parameter for rbf kernel function 
    kernel_pamR2 = pd.to_numeric(kernel_parameter[9].str.split(",", expand=True)[0])
    print(kernel_pamR2)
    
    # set up kernel matrix
    AB_signals_kernel_matrix = rbf_kernel(data, gamma= kernel_pamR2)
    AB_signals_kernel_matrix, c_list, info_list = make_symmetric_matrix_psd(AB_signals_kernel_matrix)
    
    if c_list:
        print(
            "multi_AB_signals_time_dose_kernel_matrix kernel had to be corrected.\n"
            f"model: {model}"
        )
    print("Dimension of kernel matrix with feature")
    print(AB_signals_kernel_matrix.shape)

    #kernel paramters
    #if kernel_pam.iat[0,2] == "gamma:":
    #    ABgamma = pd.to_numeric(kernel_pam.iat[0,3])
        #set up kernel matrix_rank
    #    AB_signals_kernel_matrix = rbf_kernel(data, gamma= ABgamma)
    #    AB_signals_kernel_matrix = make_svm_positive(AB_signals_kernel_matrix)
        #print("Dimension of kernel matrix with feature")
        #print(AB_signals_kernel_matrix.shape)
    #elif kernel_pam.iat[0,2] == "P:":
    #    ABdegree = pd.to_numeric(kernel_pam.iat[0,3])
        #set up kernel matrix_rank
    #    AB_signals_kernel_matrix = polynomial_kernel(data, degree=ABdegree)
    #    AB_signals_kernel_matrix = make_svm_positive(AB_signals_kernel_matrix)
        #print("Dimension of kernel matrix with feature")
        #print(AB_signals_kernel_matrix.shape)

    # set up feature test sample for distance evaluation
    test_sample = AB_signals_kernel_matrix[-1,:len(AB_signals_kernel_matrix[0])-1]

    return test_sample


In [148]:
feature_gram_matrix(proteome_data_whole.iloc[:,5:], param_combi)

2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(120, 120)


array([0.09878584, 0.02420196, 0.00363409, 0.04455595, 0.02471144,
       0.20734511, 0.03341915, 0.23391478, 0.28078507, 0.17353035,
       0.10699454, 0.20245459, 0.14715326, 0.19653973, 0.20190531,
       0.0642773 , 0.03168157, 0.22392187, 0.10434108, 0.19905111,
       0.21468414, 0.0877983 , 0.24289947, 0.22041378, 0.22034389,
       0.25634778, 0.09604983, 0.07043531, 0.27674671, 0.19492454,
       0.0173306 , 0.1214072 , 0.23108621, 0.05488571, 0.20413589,
       0.17167686, 0.20561467, 0.17840672, 0.20942686, 0.35142503,
       0.15189386, 0.01240397, 0.00546199, 0.04554244, 0.02124832,
       0.23065663, 0.02261917, 0.20212761, 0.30689972, 0.01126604,
       0.02361236, 0.09387783, 0.03562222, 0.20587885, 0.01965218,
       0.02343088, 0.02587868, 0.20311673, 0.14403334, 0.20087446,
       0.21097906, 0.11744777, 0.11825033, 0.1242474 , 0.23365451,
       0.04109096, 0.1439213 , 0.02889706, 0.21295866, 0.22309679,
       0.00610694, 0.21470268, 0.02158821, 0.04186632, 0.20256

In [149]:
def compute_distance_hyper(combinations, model, input_labels, data, kernel_paramter):
    """Evaluate distance of each single feature to classification boundary

                Compute distance of support vectors to classification boundery for each feature and the change of
                each feature by upper and lower quantile

                Args: combinations (vector): vector of combination of feature value, itself, upper and lower quantile
                      model (SVM model): multitask-SVM model
                      input-labels (list) = list of feature names
                      data (matrix): pre-processed proteome data, n x m matrix (n = samples as rows, m = features as columns)
                      kernel_parameter (dictionary): combination of kernel parameter

                Returns: get_distance_df (matrix): matrix of distance values for each feature per time point

                """

    # get labels, start with first PF-Antigen name
    labels = list(input_labels.columns.values)

    # empty array for lower and upper quantile
    get_distance_lower = []
    get_distance_upper = []

    # calc distances for all combinations
    for m in range(1, len(combinations)):
        # add test combination as new sample to
        data.loc["eval_feature", :] = combinations[m]
        single_feature_sample = feature_gram_matrix(data, kernel_paramter)
        #print(single_feature_sample.reshape(1,-1))
        distance = model.decision_function(single_feature_sample.reshape(1, -1))
        #print(distance)
        if m % 2:
            get_distance_upper.append(distance[0])
        else:
            get_distance_lower.append(distance[0])
        #print(m)
        #print(distance)

    # generate consensus feature
    data.loc["eval_feature", :] = combinations[0]
    feature_consensus_sample = feature_gram_matrix(data, kernel_paramter)
    #print(feature_consensus_sample.shape)
    # compute distance for consensus sample
    d_cons = model.decision_function(feature_consensus_sample.reshape(1, -1))
    #print(d_cons)
    # get data frame of distances values for median, lower and upper quantile
    print("Matrix of distances for Upper-/Lower- quantile per feature")
    get_distance_df = pd.DataFrame([get_distance_upper, get_distance_lower], columns=labels)
    #print(get_distance_df.shape)

    # add distance of consensus feature
    get_distance_df.loc["consensus [d]"] = np.repeat(d_cons, len(get_distance_df.columns))
    #print(get_distance_df["med"].iloc[0])
    #print(get_distance_df.shape)
    print("Number of columns of evaluated distance matrix")
    print(len(get_distance_df.columns))

    # calculate absolute distance value |d| based on lower and upper quantile
    d_value = 0
    for col in get_distance_df:
        # get evaluated distance based on upper quantile minus consensus
        val_1 = get_distance_df[col].iloc[0] - get_distance_df[col].iloc[2]
        #print(val_1)
        get_distance_df.loc["UQ - consensus [d]", col] = val_1

        # get evaluated distance based on lower quantile minus consensus
        val_2 = get_distance_df[col].iloc[1] - get_distance_df[col].iloc[2]
        #print(val_2)
        get_distance_df.loc["LQ - consensus [d]", col] = val_2

        # calculate maximal distance value from distance_based on lower quantile and distance_based on upper quantile
        if val_1 >= 0 or val_1 < 0 and val_2 > 0 or val_2 <= 0:
            a = max(abs(val_1), abs(val_2))
            if a == abs(val_1):
                d_value = val_1
            else:
                d_value = val_2

        get_distance_df.loc["|d|", col] = d_value

    # set up final data frame for distance evaluation
    get_distance_df = get_distance_df.rename({0: "UpperQuantile [d]", 1: "LowerQuantile [d]"}, axis='index')
    get_distance_df = get_distance_df.T.sort_values(by="|d|", ascending=False).T
    #sort values by abs-value of |d|
    get_distance_df.loc["sort"] = abs(get_distance_df.loc["|d|"].values)
    #print("Dimension of distance matrix")
    #print(get_distance_df.shape)
    #print("end computation")

    return get_distance_df


In [153]:
distances_for_all_feature_comb = compute_distance_hyper(all_feature_combinations, multitask_model, combinations, proteome_data_whole.iloc[:,5:], param_combi_RRR)

2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: flo

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: flo

...

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: flo

Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with feature
(121, 121)
2    0.00001
Name: 0, dtype: float64
Dimension of kernel matrix with 

In [154]:
distances_for_all_feature_comb

,PF3D7_0901000.iso1.exon1.amp1_3010,PF3D7_1124500.1o1_3341,PF3D7_0401000.iso1.exon1.amp1_2432,PF3D7_0715000.1o1_1349,PF3D7_1328300.1o3_1680,PF3D7_0317500.iso1.exon1.amp1_3396,PF3D7_1035900.1o1_1406,PF3D7_0900500.iso1.exon1.amp1_4464,PF3D7_0222700.iso1.exon2.amp1_1289,PF3D7_0632100.iso1.exon1.amp1_705,...,PF3D7_0915800.1o1_2637,PF3D7_0707800.e1s2_1501,PF3D7_0425400.2o2_46,PF3D7_1250100.e1s1_2231,PF3D7_1004500.1o1_43,PF3D7_1446900.2o3_80,PFB0300c_4064,PF3D7_1372800.iso1.exon2.amp1_3009,PF3D7_0400800.iso1.exon1.amp1_3008,PF3D7_0402400.2o2_48
UpperQuantile [d],0.763728,0.762903,0.762723,0.762536,0.762251,0.762147,0.762047,0.762011,0.761745,0.761648,...,0.751362,0.751263,0.751248,0.751126,0.751106,0.751023,0.758287,0.750666,0.750391,0.749348
LowerQuantile [d],0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,...,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.750748,0.755989,0.755989,0.755989
consensus [d],0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,...,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989,0.755989
UQ - consensus [d],0.007739,0.006914,0.006734,0.006547,0.006262,0.006158,0.006058,0.006022,0.005756,0.005659,...,-0.004627,-0.004726,-0.004741,-0.004863,-0.004883,-0.004966,0.002298,-0.005323,-0.005598,-0.006641
LQ - consensus [d],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.005241,0.000000,0.000000,0.000000
|d|,0.007739,0.006914,0.006734,0.006547,0.006262,0.006158,0.006058,0.006022,0.005756,0.005659,...,-0.004627,-0.004726,-0.004741,-0.004863,-0.004883,-0.004966,-0.005241,-0.005323,-0.005598,-0.006641
sort,0.007739,0.006914,0.006734,0.006547,0.006262,0.006158,0.006058,0.006022,0.005756,0.005659,...,0.004627,0.004726,0.004741,0.004863,0.004883,0.004966,0.005241,0.005323,0.005598,0.006641
